<a target="_blank" href="https://colab.research.google.com/github/sergiopaniego/RAG_local_tutorial/blob/main/youtube_rag.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Example of RAG with a Youtube video

Here, we download the trascription of a Youtube video and use an LLM for extracting information from that video.


<p align="center">
  <img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2023/07/langchain3.png" alt="Langchain Logo" width="20%">
  <img src="https://bookface-images.s3.amazonaws.com/logos/ee60f430e8cb6ae769306860a9c03b2672e0eaf2.png" alt="Ollama Logo" width="20%">
  <img src="https://upload.wikimedia.org/wikipedia/commons/e/ef/Youtube_logo.png" alt="Youtube Logo" width="20%">
</p>


# Requirements

* Ollama installed locally

### Install the dependencies

In [ ]:
!pip3 install langchain
!pip3 install langchain_pinecone
!pip3 install langchain[docarray]
!pip3 install docarray
!pip3 install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.15
    Uninstalling aiohttp-3.11.15:
      Successfully uninstalled aiohttp-3.11.15
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip3 install youtube_transcript_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.6 MB/s eta 0:00:00


In [ ]:
!pip install ollama

In [ ]:
!apt install pciutils -y
!curl -fsSL https://ollama.com/install.sh | sh
!ollama run llama3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pci.ids all 0.0~2022.01.22-1ubuntu0.1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (350 kB/s)
Selecting previously unselected package pci.ids.
(Reading database ... 126333 files and directories currently installed.)
Preparing to unpack .../pci.ids_0.0~2022.01.22-1ubuntu0.1_all.deb ...
Unpacking pci.ids (0.0~2022.01.22-1ubuntu0.1) ...
Selecting previously unsele

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00


# Let's download an example transcript from a YT video

You can change the id of the video to download other video transcriptions.
We save the contect to a file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from youtube_transcript_api import YouTubeTranscriptApi

srt = YouTubeTranscriptApi.get_transcript("pxiP-HJLCx0") # CHANGE THE ID OF THE VIDEO

with open("/content/drive/MyDrive/AIMLTraining/LangChain/files/youtube_transcription.txt", "a") as file:
    for i in srt:
        print(i['text'])
        file.write(i['text'])

Mounted at /content/drive
in this video I'm going to tell you the
best laptops for students now for this
one my team and I went absolutely nuts
we got in pretty much every viable
student laptop think I'm joking I am not
we tested an epic 15 laptops everything
from Apple's MacBook Air to Microsoft
Surface laptop from Asus zenbooks to
Samsung Galaxy books you name it we
tested it oh and if you're wondering
what our experience with student laptops
is between the three of us who worked on
this video we have a combined five
University degrees now I value your time
so I'm going to very first briefly hit
on some important points that you must
be aware of when picking a laptop for
school then I'm going to get straight
into what you're actually here to see
which is which laptops we recommend for
various types of students firstly the
laptop should be small and portable
unless we get hit by another virus which
we all hope we don't students frequently
move from classroom to classroom plus
they oft

# Select the LLM model to use

The model must be downloaded locally to be used, so if you want to run llama3, you should run:

```

ollama pull llama3

```

Check the list of models available for Ollama here: https://ollama.com/library

# We instantiate the model and the embeddings

In [ ]:
#MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
#MODEL = "gemma:7b"
#MODEL = "llama2"
MODEL = "llama3" # https://ollama.com/library/llama3

In [ ]:
import os
import asyncio

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

In [ ]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

>>> starting ollama serve


In [ ]:
!ollama pull llama3

[GIN] 2025/04/27 - 16:32:02 | 200 |     129.256µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ time=2025-04-27T16:32:03.153Z level=INFO source=download.go:177 msg="downloading 6a0746a1ec1a in 16 291 MB part(s)"
pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 6a0746a1ec1a:   1% ▕▏  34 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   3% ▕▏ 158 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   5% ▕▏ 222 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   8% ▕▏ 364 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:  10% ▕▏ 487 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:  12% ▕▏ 561 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:  15% ▕▏ 684 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:  18% ▕▏ 821 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

<ipython-input-11-5c36d39e2206>:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model=MODEL)
<ipython-input-11-5c36d39e2206>:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=MODEL)


# We load the transcription previously saved using TextLoader

In [ ]:
from langchain_community.document_loaders import TextLoader
# let colab access my google drive
from google.colab import drive
drive.mount('/content/drive')

loader = TextLoader("/content/drive/MyDrive/AIMLTraining/LangChain/files/youtube_transcription.txt")
text_documents = loader.load()
text_documents

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[Document(metadata={'source': '/content/drive/MyDrive/AIMLTraining/LangChain/files/youtube_transcription.txt'}, page_content="in this video I'm going to tell you thebest laptops for students now for thisone my team and I went absolutely nutswe got in pretty much every viablestudent laptop think I'm joking I am notwe tested an epic 15 laptops everythingfrom Apple's MacBook Air to MicrosoftSurface laptop from Asus zenbooks toSamsung Galaxy books you name it wetested it oh and if you're wonderingwhat our experience with student laptopsis between the three of us who worked onthis video we have a combined fiveUniversity degrees now I value your timeso I'm going to very first briefly hiton some important points that you mustbe aware of when picking a laptop forschool then I'm going to get straightinto what you're actually here to seewhich is which laptops we recommend forvarious types of students firstly thelaptop should be small and portableunless we get hit by another virus whichwe all hop

# We explit the document into chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)
text_documents = text_splitter.split_documents(text_documents)[:]
text_documents

[Document(metadata={'source': '/content/drive/MyDrive/AIMLTraining/LangChain/files/youtube_transcription.txt'}, page_content="in this video I'm going to tell you thebest laptops for students now for thisone my team and I went absolutely nutswe got in pretty much every viablestudent laptop think I'm joking I am notwe tested an epic 15 laptops everythingfrom Apple's MacBook Air to MicrosoftSurface laptop from Asus zenbooks toSamsung Galaxy books you name it wetested it oh and if you're wonderingwhat our experience with"),
 Document(metadata={'source': '/content/drive/MyDrive/AIMLTraining/LangChain/files/youtube_transcription.txt'}, page_content="our experience with student laptopsis between the three of us who worked onthis video we have a combined fiveUniversity degrees now I value your timeso I'm going to very first briefly hiton some important points that you mustbe aware of when picking a laptop forschool then I'm going to get straightinto what you're actually here to seewhich is whi

In [ ]:
text_documents

[Document(metadata={'source': '/content/drive/MyDrive/AIMLTraining/LangChain/files/youtube_transcription.txt'}, page_content="in this video I'm going to tell you thebest laptops for students now for thisone my team and I went absolutely nutswe got in pretty much every viablestudent laptop think I'm joking I am notwe tested an epic 15 laptops everythingfrom Apple's MacBook Air to MicrosoftSurface laptop from Asus zenbooks toSamsung Galaxy books you name it wetested it oh and if you're wonderingwhat our experience with"),
 Document(metadata={'source': '/content/drive/MyDrive/AIMLTraining/LangChain/files/youtube_transcription.txt'}, page_content="our experience with student laptopsis between the three of us who worked onthis video we have a combined fiveUniversity degrees now I value your timeso I'm going to very first briefly hiton some important points that you mustbe aware of when picking a laptop forschool then I'm going to get straightinto what you're actually here to seewhich is whi

# Store the PDF in a vector space.

From Langchain docs:

`DocArrayInMemorySearch is a document index provided by Docarray that stores documents in memory. It is a great starting point for small datasets, where you may not want to launch a database server.`

The execution time of the following block depends on the complexity and longitude of the PDF provided. Try to keep it small and simple for the example.

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(text_documents, embedding=embeddings)

time=2025-04-27T16:39:26.145Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-04-27T16:39:26.145Z level=WARN source=types.go:570 msg="invalid option provided" option=tfs_z
[GIN] 2025/04/27 - 16:39:26 | 200 |   68.687444ms |       127.0.0.1 | POST     "/api/embeddings"
time=2025-04-27T16:39:26.220Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-04-27T16:39:26.220Z level=WARN source=types.go:570 msg="invalid option provided" option=tfs_z
[GIN] 2025/04/27 - 16:39:26 | 200 |   68.355182ms |       127.0.0.1 | POST     "/api/embeddings"
time=2025-04-27T16:39:26.289Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-04-27T16:39:26.290Z level=WARN source=types.go:570 msg="invalid option provided" option=tfs_z
[GIN] 2025/04/27 - 16:39:26 | 200 |    64.48676ms |       127.0.0.1 | POST     "/api/embeddings"
time=2025-04-27T16:39:26.359Z level=WARN source=ggml.go:15

In [ ]:
retriever = vectorstore.as_retriever()

# We instantiate the parser

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# Generate the conversation template

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, answer with "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t\nanswer the question, answer with "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

# We can now extract the information from the video!

In [ ]:
retrieved_context = retriever.invoke("laptop")
questions = [
    "Which are the most important considerations when buying a laptop?",
    "How much is a laptop worth?",
    "Make a summary of the text"
]

for question in questions:
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = model.invoke(formatted_prompt)
    parsed_response = parser.parse(response_from_model)

    print(f"Question: {question}")
    print(f"Answer: {parsed_response}")
    print()

time=2025-04-27T16:41:19.206Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-04-27T16:41:19.207Z level=WARN source=types.go:570 msg="invalid option provided" option=tfs_z
[GIN] 2025/04/27 - 16:41:19 | 200 |   41.842622ms |       127.0.0.1 | POST     "/api/embeddings"
time=2025-04-27T16:41:19.274Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-04-27T16:41:19.279Z level=WARN source=types.go:570 msg="invalid option provided" option=tfs_z
[GIN] 2025/04/27 - 16:41:20 | 200 |  1.115729202s |       127.0.0.1 | POST     "/api/generate"
Question: Which are the most important considerations when buying a laptop?
Answer: Based on the context, the most important considerations when buying a laptop mentioned in the document are:

* A resolution higher than 1920x1200
* A screen brighter than 400nits to see the screen in bright environments like using it close to a window
* Enough performance to be future-proo